In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!python -V

Python 3.10.13


In [3]:

import pickle
import pandas as pd
import sys

In [18]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
year = int(sys.argv([1]))
month = int(sys.argv([2]))
save = sys.argv[3]

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
    return df

In [15]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [16]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [17]:
y_pred.mean()

13.961569832229767

In [11]:
def result_df(df):
    df_2 = pd.DataFrame()
    df_2['ride_id'] = df['ride_id']
    df_2['predicted'] = y_pred

    return df_2

In [13]:
result_df = result_df(df)

,ride_id,predicted
0,2022/02_0,19.415866
1,2022/02_1,26.067702
2,2022/02_2,35.530950
3,2022/02_3,23.261276
4,2022/02_4,24.529900
...,...,...
2979426,2022/02_2979426,13.354209
2979427,2022/02_2979427,11.610930
2979428,2022/02_2979428,13.115060
2979429,2022/02_2979429,14.465821


In [14]:
result = {'mean duration' : result_df.mean(), 'std duration': result_df.std()}

In [ ]:
if __name__ == "__main__":
    result